Trying to replicate models developed by David Thaler:

https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/forums/t/8125/first-place-entry/56111#post56111

https://bitbucket.org/dthal/kaggle_walmart

In [221]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from IPython.display import display, HTML
%matplotlib inline


In [336]:
'''
For now, just get models working on subset of data, 
for which sales are available on every day of the observation period

Note: Only 6 of these 10 also appear in the test data
'''
# 10 stores to test models on: [769, 1097, 85, 562, 262, 733, 494, 682, 335, 423]
id_list = [769, 1097, 85, 562, 262, 733, 494, 682, 335, 423]

data = pd.read_csv('train.csv', parse_dates = ['Date'])
test_raw = pd.read_csv('test.csv', parse_dates = ['Date'])

train = data[data.Store.isin(set(id_list))][['Store','DayOfWeek','Date','Sales','Promo',
                                             'StateHoliday','SchoolHoliday']]

test = test_raw[test_raw.Store.isin(set(id_list))][['Store','DayOfWeek','Date','Promo',
                                             'StateHoliday','SchoolHoliday']]

In [364]:
train.describe();

## Utility and misc functions:

In [392]:
# Keeping potential util functions in this cell
def daily_mean(train, years=None, yr_target=None, lag=None):
    '''
    Computes avg daily sales across all years provided. User-specified lag
    period defines the length of seasonality to account for when calculating the mean.
    
    INPUT:
    train - dataframe of data, with cols 'Sales', 'Date', 'Store'
    years - list of years over which the mean should be calculated
    yr_target - int, calendar year of desired output 
    lag_days - int, number of days to lag when shifting between each year
               this field ensures that we're calculating the mean across 
               equivalent weekdays for the year specified by yr_target
    
    For each year:
        Shift date index to yr_target (2015), using 364 day lag
        Create pivot table from shifted data (shifted Date x Store)
        Add to dict
    Convert to panel data structure (Year x shifted Date x Store)
    Calculate mean over year index (returns mean for 'same' weekday from each year)
    '''
    if years is None:
        years = [2013, 2014, 2015]
    if yr_target is None:
        yr_target = 2015
    if lag is None:
        lag = 364
        
    panel_dict = dict.fromkeys(years)

    train_copy = train.loc[:,['Sales','Date','Store']].set_index('Date')

    for yr in years:
        # Create copy of training data
        # Shift date forward to match the forecast year:
        # Each year should actually be shifted by 364 days to ensure that the 
        # equivalent weekday is preserved. DOES NOT ACCOUNT FOR LEAP YEARS
        temp = train_copy.shift((yr_target - yr) * lag, pd.datetools.day)

        
        # Add to dictionary as pivot table of Sales (shifted Date x store)
        # Pull only rows with shifted Date.year = yr_target
        if np.sum(temp.index.year == yr_target) > 0:
            panel_dict[yr] = pd.pivot_table(temp.loc[temp.index.year == yr_target], 
                                            values='Sales', 
                                            index = temp.loc[temp.index.year == yr_target].index,
                                            columns='Store', 
                                            aggfunc=np.mean)
            
    # return mean of panel created from dictionary
    return pd.Panel(panel_dict).mean(axis=0)

## Time Series Models:

In [361]:
# Keeping time series models in this cell
def naive(train, test):
    '''
    Computes naive forecasts (forecast = last observed value)
    
    INPUT:
    train - dataframe of daily sales values [1 row for each (date, Store) duple]
    test - dataframe of daily store information [1 row for each (date, Store) duple]
    
    OUTPUT:
    out - dataframe of test data with columns: date, Store, Forecasted Sales
    '''
    # Make a copy of the test dataframe to use for forecasting
    out = test[['Store','Date']].set_index('Date')
    
    # Subset only the last observation from the train dataframe
    # The sales on this day will be used for the naive forecast
    tr = train.loc[train.Date==train.Date.max(),:].set_index('Store')
    
    # Apply the last know sales value to each forecast date in the
    # test data
    out.loc[:,'SalesForecast'] = out.Store.map(tr.Sales).values
    return out

def seasonal_naive(train, test):
    '''
    Computes seasonal naive forecasts (forecast = last observed value for same
    seasonal unit). In this case, seasonality is approximated by a 364 day lag,
    in order to ensure that the day of the week is preserved.
    
    INPUT:
    train - dataframe of daily sales values [1 row for each (date, Store) duple]
    test - dataframe of daily store information [1 row for each (date, Store) duple]
    
    OUTPUT:
    out - dataframe of test data with columns: date, Store, Forecasted Sales
    '''
    lag = 364 # lag by 364 days (won't work in leap year)
    
    # Make a copy of the test dataframe to use for forecasting
    out = test[['Store','Date','Promo']].set_index(['Date','Store'])
    
    # Reset date index to lag data by +364 days. This shift ensures that
    # the test data is set to the same weekday from 1 year ago
    # (does not take into account leap years)
    tr_lag = train[['Sales','Date','Store']].set_index('Date').shift(lag, pd.datetools.day)
    tr_lag.set_index('Store',append=True, inplace=True)
    
    # Merge data, keeping only daterange from test data
    out = out.merge(pd.DataFrame(tr_lag), how='left', left_index=True, right_index=True).drop('Promo',axis=1)
    out.columns = ['SalesForecast']
    
    return out

def product(train, test):
    '''
    Computes forecasts with the product model. This model predicts the mean
    value by store times the mean value by day divided by the overall mean
    of daily sales. Once again, the 'mean value by day' needs to make sure that
    the day of week is preserved.
    
    INPUT:
    train - dataframe of daily sales values [1 row for each (date, Store) duple]
    test - dataframe of daily store information [1 row for each (date, Store) duple]
    
    OUTPUT:
    out - dataframe of test data with columns: date, Store, Forecasted Sales
    '''
    
    return out

In [362]:
out = naive(train, test)
out = seasonal_naive(train, test)

In [363]:
out.head()

SalesForecast
Date       Store               
2015-09-17 262            16660
           335            16747
           562            16289
           733            15415
           769            11083

In [195]:
out.head()

SalesForecast
Date       Store               
2015-09-17 262            16660
           335            16747
           562            16289
           733            15415
           769            11083

# Scratchwork
Scratchwork below this point, to test out code as I go

In [116]:
panel_dict = dict.fromkeys(id_list)

train_copy = train.set_index('Date')
for store in id_list:
    panel_dict[store] = train_copy.query('Store == @store')
    
panel_train = pd.Panel(panel_dict)
panel_train


<class 'pandas.core.panel.Panel'>
Dimensions: 10 (items) x 942 (major_axis) x 6 (minor_axis)
Items axis: 85 to 1097
Major_axis axis: 2015-07-31 00:00:00 to 2013-01-01 00:00:00
Minor_axis axis: Store to SchoolHoliday

In [222]:
ct = pd.pivot_table(train, values='Sales', index='Date', 
                            columns='Store', aggfunc=np.mean)
display( ct.head(3) )

ct_lag = ct.iloc[-364:,:]
display( ct_lag.head(3) )

ct_copy = ct_lag.unstack().reset_index()
ct_copy.Date = ct_copy.Date + np.timedelta64(364,'D')
ct_copy.set_index(['Date','Store'], inplace=True)
display( ct_copy.head(3) )

Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2013-01-01,4220,17267,2401,9643,3113,8498,3375,10765,5035,5961
2013-01-02,6069,16964,11542,9570,6300,15472,10526,12477,7276,6688
2013-01-03,5246,16616,10686,8254,6209,14807,11041,12639,6972,7053


Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2014-08-02,5845,19920,9115,9331,5922,14427,6985,15521,10893,7112
2014-08-03,12265,30097,8721,14633,8759,21206,6726,16139,11572,12606
2014-08-04,8779,20103,19878,11439,10004,20483,12031,16785,10955,9826


,,0
Date,Store,
2015-08-01,85,5845
2015-08-02,85,12265
2015-08-03,85,8779


In [332]:
'''
For each year:
    Shift date index to 2015, using 364 day lag to account for each year shifted
    Create pivot table from shifted data (shifted Date x Store)
    Add to dict
Convert to panel data structure (Year x shifted Date x Store)
Calculate mean over year index (returns mean for 'same' weekday from each year)
'''
lag = 364
years = [2013, 2014, 2015]
yr_target = 2015
panel_dict = dict.fromkeys(years)

train_copy = train.loc[:,['Sales','Date','Store']].set_index('Date')

for yr in years:
    # Create copy of training data
    # Shift date forward to match the forecast year:
    # Each year should actually be shifted by 364 days to ensure that the 
    # equivalent weekday is preserved. DOES NOT ACCOUNT FOR LEAP YEARS
    temp = train_copy.shift(lag, pd.datetools.day)
    
    # Add to dictionary as pivot table of Sales (shifted Date x store)
    # Pull only rows with shifted Date.year = yr_target
    panel_dict[yr] = pd.pivot_table(temp.loc[temp.index.year == yr_target], 
                                    values='Sales', 
                                    index = temp.loc[temp.index.year == yr_target].index,
                                    columns='Store', 
                                    aggfunc=np.mean)

# Create panel from dictionary
panel_yrly = pd.Panel(panel_dict)
panel_yrly.mean(axis=0).describe()

,85,262,335,423,494,562,682,733,769,1097
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,7265.241096,20659.323288,13457.416438,10644.838356,7701.131507,18017.621918,11052.693151,15165.331507,11180.482192,9829.912329
std,2215.260595,4761.598672,4897.202895,2134.252659,1693.902967,2984.700850,3216.864074,1865.948476,1589.116040,1918.334826
min,3609.000000,13210.000000,3428.000000,5886.000000,3802.000000,10552.000000,4423.000000,6838.000000,7096.000000,5767.000000
25%,5686.000000,17217.000000,9273.000000,9195.000000,6415.000000,16055.000000,8174.000000,13942.000000,10125.000000,8649.000000
50%,6665.000000,19386.000000,12579.000000,10346.000000,7640.000000,17967.000000,10811.000000,15210.000000,11041.000000,9656.000000
75%,8121.000000,22128.000000,17081.000000,11713.000000,8842.000000,19890.000000,13226.000000,16282.000000,12046.000000,10895.000000
max,15386.000000,37403.000000,31406.000000,17317.000000,13309.000000,27716.000000,20230.000000,21623.000000,17200.000000,16304.000000


In [240]:
display(panel_yrly.mean(axis=0).head(1) )
display(panel_yrly[2015].head(1) )
display(panel_yrly[2014].head(1) )
display(panel_yrly[2013].head(1) )

Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2015-01-01,5346.666667,18458.666667,8421,9122,5375,13553.666667,8635.333333,13282.666667,7906.333333,7534.333333


Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2015-01-01,4576,20581,3428,9997,3802,10552,4506,12566,7538,7836


Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2015-01-01,6218,18179,11149,9115,6114,15302,10359,14643,9209,7714


Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2015-01-01,5246,16616,10686,8254,6209,14807,11041,12639,6972,7053


In [393]:
daily_mean(train).head()

Store,85,262,335,423,494,562,682,733,769,1097
Date,,,,,,,,,,
2015-01-01,5346.666667,18458.666667,8421.000000,9122.000000,5375.000000,13553.666667,8635.333333,13282.666667,7906.333333,7534.333333
2015-01-02,5814.000000,17034.000000,10676.333333,9679.000000,6642.333333,15026.333333,10307.666667,13342.666667,8829.333333,7058.666667
2015-01-03,5941.000000,15765.333333,8834.000000,7502.000000,5101.666667,12994.000000,7744.000000,12283.333333,8391.000000,6119.666667
2015-01-04,10310.333333,23524.666667,7826.000000,10690.333333,7192.666667,17375.666667,6691.666667,13629.000000,8992.000000,10368.333333
2015-01-05,8621.666667,19981.666667,21430.000000,10520.666667,10816.666667,22760.000000,16757.666667,14698.000000,10971.000000,9878.000000


In [211]:
(train.Date + np.timedelta64(364,'D')).iloc[3630]

Timestamp('2015-08-01 00:00:00')

In [262]:
train_copy = train.loc[:,['Date','Store','Sales']].set_index('Date')
display(train_copy.head())

train_shifted = train_copy.shift(364, freq=pd.datetools.day)
display(train_shifted.head())

,Store,Sales
Date,,
2015-07-31,85,7791
2015-07-31,262,25774
2015-07-31,335,17867
2015-07-31,423,13331
2015-07-31,494,9970


,Store,Sales
Date,,
2016-07-29,85,7791
2016-07-29,262,25774
2016-07-29,335,17867
2016-07-29,423,13331
2016-07-29,494,9970


In [253]:
test = pd.DatetimeIndex(['2013-01-01','2014-01-01','2015-01-01'])
test.dayofweek

array([1, 2, 3], dtype=int32)

In [255]:
test = pd.DatetimeIndex(['2013-01-01','2014-01-01','2015-01-01'])
print test
print test.dayofweek

test_shift = test + np.timedelta64(364,'D')
print test_shift
print test_shift.dayofweek

test_shift = test_shift + np.timedelta64(364,'D')
print test_shift
print test_shift.dayofweek

DatetimeIndex(['2013-01-01', '2014-01-01', '2015-01-01'], dtype='datetime64[ns]', freq=None, tz=None)
[1 2 3]
DatetimeIndex(['2013-12-31', '2014-12-31', '2015-12-31'], dtype='datetime64[ns]', freq='A-DEC', tz=None)
[1 2 3]
DatetimeIndex(['2014-12-30', '2015-12-30', '2016-12-29'], dtype='datetime64[ns]', freq='365D', tz=None)
[1 2 3]


In [257]:
test = pd.DatetimeIndex(['2013-01-03','2014-01-02','2015-01-01'])
print test
print test.dayofweek

test_shift = test + np.timedelta64(364,'D')
print test_shift
print test_shift.dayofweek

test_shift = test_shift + np.timedelta64(364,'D')
print test_shift
print test_shift.dayofweek

DatetimeIndex(['2013-01-03', '2014-01-02', '2015-01-01'], dtype='datetime64[ns]', freq=None, tz=None)
[3 3 3]
DatetimeIndex(['2014-01-02', '2015-01-01', '2015-12-31'], dtype='datetime64[ns]', freq='52W-THU', tz=None)
[3 3 3]
DatetimeIndex(['2015-01-01', '2015-12-31', '2016-12-29'], dtype='datetime64[ns]', freq='52W-THU', tz=None)
[3 3 3]


In [ ]:
test = pd.DatetimeIndex(['2013-01-03','2014-01-02','2015-01-01'])
print test
print test.dayofweek

test_shift = test + np.timedelta64(364,'D')
print test_shift
print test_shift.dayofweek